## The Cython Project

[Cython](https://cython.org/) ("C extensions for Python") is intended as a superset of the Python language, embedding it in a C-based framework that allows to increase the performance of the code by compiling it. Here, we will only look at the usage of Cython within a jupyter notebook, as this is particularly easy.

We will reuse the same example function, the mean of the difference of two vectors, from the [Numba notebook](node15_numba.ipynb) to later compare the performance, so let us first define the input vectors again:

In [ ]:
# as before: generate two large vectors
import numpy as np
l = 10000
x = np.random.randn(l)
y = np.random.randn(l)

Using again NumPy arrays as returned by these functions rather than e.g. Python lists has the advantage that they internally use a very low-level (and contiguous) memory representation so that we can later pass them to the Cython function without having to convert them:

In [ ]:
type(x)

In [ ]:
# first compute with numpy to see what result we should expect:
(x-y).mean()

As before we need to install and load Cython:

In [ ]:
# cython must have been installed
%load_ext Cython
# also may need to set include path for C compiler before starting jupyter, e.g.
#   export CPATH=/usr/include/python3.5m/
# the path must include Python.h

By adding the `%%cython` magic cell command, we can tell the jupyter notebook to compile the cell content into a standalone Cython module. The `-a` switch gives us an annotated output that is very useful for debugging and optimization. Lines marked in yellow tell us where interaction with Python (the more (and darker) yellow, the slower the code) occurs, and we can click on any line to see the generated C code.

In [ ]:
%%cython -a
def compute_mean_distance_cython(x, y):
    # computes mean distance over 2 vectors
    l = len(x)
    s = 0
    for idx in range(l):
        s = s + (x[idx]-y[idx])
    return s / l


In [ ]:
# does it work?
compute_mean_distance_cython(x, y)

In [ ]:
%%timeit
compute_mean_distance_cython(x, y)

This is already a bit faster -- but not really optimized as we can see from the many yellow lines above. Let's try to improve. For that we use the new command `cdef` to specify the C-types of variables and pass the input vectors as C arrays (["typed memory views"](https://cython.readthedocs.io/en/latest/src/userguide/memoryviews.html)). We will also disable the bounds checking (although the overhead from this is rather small, as you can test for yourself) with the decorator `boundscheck`.

In [ ]:
%%cython -a
from cython cimport boundscheck
@boundscheck(False)
def compute_mean_distance_cython(const double[:] x, const double[:] y):
    # computes mean distance over 2 vectors
    cdef double s = 0
    cdef long l = len(x)
    for idx in range(l):
        s = s + (x[idx]-y[idx])
    return s / l

In [ ]:
compute_mean_distance_cython(x, y)

In [ ]:
%%timeit
compute_mean_distance_cython(x, y)

This is pretty much as fast as Numba, but was certainly somewhat more effort to arrive at. In both cases this is just a very quick glimpse at what the packages can do so that you know they exist and what they can do for you. To get started with a real-life application you will need to dive deeper into the details of the [documentation](https://cython.readthedocs.io/en/latest/).